In [2]:
import constraint
import numpy as np
import copy

In [3]:

class ModelParser:
    @staticmethod
    def var_ix(vs):
        assert vs[0] == 'X', f"Bad vs {vs}"
        return int(vs[1:])
    @staticmethod
    def is_var(vs):
        return isinstance(vs, str)

    def __init__(self, model, var_domain=None):
        self.var_domain = var_domain
        rows = filter(bool, map(str.strip, model.split('\n')))
        mrep = []
        for row in rows:    
            mrep.append(list(filter(bool, map(str.strip, row.split()))))

        shape = len(mrep)-2, len(mrep[0])-1
        assert shape[0] == shape[1], f"Bad shape - {shape}"

        # to ints
        var_index = 0
        for i in range(len(mrep)):    
            for j in range(len(mrep[i])):
                if mrep[i][j] == 'X':
                    mrep[i][j] = f'X{var_index}'
                    var_index += 1
                else:
                    mrep[i][j] = int(mrep[i][j])
        self.num_vars = var_index
        
        # validate all numbers in domain
        if self.var_domain:
            for row in range(shape[0]):
                for col in range(shape[1]):
                    if not self.is_var(mrep[row][col]):
                        assert mrep[row][col] in self.var_domain, f"Value {mrep[row][col]} at [{row}, {col}] is not in domain {self.var_domain}"
                    
        constraints = []
        # row sum eqns
        for i in range(shape[0]):
            row_sum = mrep[i][-1]
            var_list = []
            for v in mrep[i][:-1]:
                if self.is_var(v):
                    var_list.append(self.var_ix(v))
                else:
                    row_sum -= v
            if len(var_list):
                constraints.append((var_list, row_sum))

        # col sum eqns
        for col in range(shape[1]):
            col_sum = mrep[-2][col]
            var_list = []
            for row in range(shape[0]): 
                v = mrep[row][col]
                if self.is_var(v):
                    var_list.append(self.var_ix(v))
                else:
                    col_sum -= v
            if len(var_list):
                constraints.append((var_list, col_sum))

        # diagonal constraints - D1
        var_list = []
        d_sum = mrep[-1][0]
        for i in range(shape[0]):
            row, col = i, i            
            v = mrep[row][col]
            if self.is_var(v):
                var_list.append(self.var_ix(v))
            else:
                d_sum -= v
        if len(var_list):
            constraints.append((var_list, d_sum))


        # diagonal constraints - D2
        var_list = []
        d_sum = mrep[-1][1]
        for i in range(shape[0]):
            row, col = i, shape[1] - i - 1            
            v = mrep[row][col]
            if self.is_var(v):
                var_list.append(self.var_ix(v))
            else:
                d_sum -= v
        if len(var_list):
            constraints.append((var_list, d_sum))
        
        self.mrep = mrep
        self.constraints = constraints
        self.shape = shape
    
    def get_num_vars(self):
        return self.num_vars
    
    def print_solution(self, s):
        new_mrep = copy.deepcopy(self.mrep)

        for row in range(self.shape[0]):
            for col in range(self.shape[1]):
                if self.is_var(new_mrep[row][col]):
                    var_ix = self.var_ix(new_mrep[row][col])
                    new_mrep[row][col] = f'*{s[var_ix]}*'
        self.display_mrep(new_mrep)
    
    def display_mrep(self, mrep=None):
        if mrep is None:
            mrep = self.mrep
        for row in range(len(mrep)):
            row_txt = "  ".join("{:6}".format(str(mrep[row][col])) for col in range(len(mrep[row])))
            print(row_txt)
                





In [4]:
model = '''
10  10  X   X   25  60
X    X  X   X   X   55
5   10  X   0   X   35
X   X   5   X   X   25
X   10  X   X   X   50
40  55  35  35  60
65  55
'''


m = ModelParser(model, [0,5,10,25])
m.display_mrep()

10      10      X0      X1      25      60    
X2      X3      X4      X5      X6      55    
5       10      X7      0       X8      35    
X9      X10     5       X11     X12     25    
X13     10      X14     X15     X16     50    
40      55      35      35      60    
65      55    


In [5]:
m.constraints

[([0, 1], 15),
 ([2, 3, 4, 5, 6], 55),
 ([7, 8], 20),
 ([9, 10, 11, 12], 20),
 ([13, 14, 15, 16], 40),
 ([2, 9, 13], 25),
 ([3, 10], 25),
 ([0, 4, 7, 14], 30),
 ([1, 5, 11, 15], 35),
 ([6, 8, 12, 16], 35),
 ([3, 7, 11, 16], 55),
 ([5, 7, 10, 13], 30)]

In [6]:
%%time

p = constraint.Problem(constraint.BacktrackingSolver())
p.addVariables(list(range(m.num_vars)), m.var_domain)
for var_list, total in m.constraints:
    p.addConstraint(constraint.ExactSumConstraint(total), var_list)        
for s in p.getSolutions():
    print('Solution: ')
    m.print_solution(s)

Solution: 
10      10      *10*    *5*     25      60    
*10*    *25*    *0*     *10*    *10*    55    
5       10      *10*    0       *10*    35    
*5*     *0*     5       *10*    *5*     25    
*10*    10      *10*    *10*    *10*    50    
40      55      35      35      60    
65      55    
Wall time: 8 ms
